<a href="https://colab.research.google.com/github/misrori/goldhandtrading/blob/main/dash_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install goldhand dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=bbd6809e772fca68710443c9f6673367983b03574fe2e08dae06ffa2cc72b1f3
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from goldhand import *

tw = Tw()
df = tw.stock

# Choose the column you want to display in the dropdown
name_list = list(zip(list(df['description']), list(df['name'])))

# Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='name-dropdown',
        options=[{'label': x[0], 'value': x[1]} for x in name_list],
        value=name_list[0][1]  # Set the default selected value
    ),
    html.Button('Previous', id='prev-button', n_clicks=0),
    html.Button('Next', id='next-button', n_clicks=0),
    dcc.Input(
        id='ndays-input',
        type='number',
        value=530,  # Set the default value
        step=10
    ),
    dcc.Graph(id='live-update-graph'),

])

@app.callback(
    [Output('live-update-graph', 'figure'),
     Output('name-dropdown', 'value')],
    [Input('name-dropdown', 'value'),
     Input('next-button', 'n_clicks'),
     Input('prev-button', 'n_clicks'),
     Input('ndays-input', 'value')]
)
def update_graph(selected_name, next_clicks, prev_clicks, ndays):
    # Determine which button was clicked
    ctx = dash.callback_context
    triggered_id = ctx.triggered_id

    # Get the current name index
    current_index = name_list.index((df.loc[df['name'] == selected_name, 'description'].values[0], selected_name))

    # Handle button clicks
    if triggered_id == 'next-button':
        next_index = current_index + 1
    elif triggered_id == 'prev-button':
        next_index = current_index - 1
    else:
        next_index = current_index

    # Ensure the index is within the valid range
    next_index = next_index % len(name_list)

    # Get the next name
    next_name = name_list[next_index][1]

    # Get the plot for the next name
    t = GoldHand(next_name)
    fig = t.plotly_last_year(plot_title=tw.get_plotly_title(next_name), plot_height=1000, ndays=ndays)

    return fig, next_name

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode='external')


Dash app running on:


<IPython.core.display.Javascript object>